# Solutions for Services Quiz

<img src="../img/robotignite_logo_text.png"/>

## Solution Services Quiz

* Upgrade the python file **move_bb8.py** so that it can move BB8 in a square of variable size.

<p style="background:#3B8F10;color:white;" id="move_bb8_py_mod">**Python File: move_bb8.py** </p>

In [ ]:
#!/usr/bin/env python
import rospy
from geometry_msgs.msg import Twist
import time
import math

class MoveBB8():
    
    def __init__(self):
        self.bb8_vel_publisher = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.ctrl_c = False
        rospy.on_shutdown(self.shutdownhook)
        self.rate = rospy.Rate(10) # 10hz
    
    
    
    def publish_once_in_cmd_vel(self, cmd):
        """
        This is because publishing in topics sometimes fails teh first time you publish.
        In continuos publishing systems there is no big deal but in systems that publish only
        once it IS very important.
        """
        while not self.ctrl_c:
            connections = self.bb8_vel_publisher.get_num_connections()
            if connections > 0:
                self.bb8_vel_publisher.publish(cmd)
                rospy.loginfo("Cmd Published")
                break
            else:
                self.rate.sleep()
    
    
    def shutdownhook(self):
            # works better than the rospy.is_shut_down()
            self.stop_bb8()
            self.ctrl_c = True

    def stop_bb8(self):
        rospy.loginfo("shutdown time! Stop the robot")
        cmd = Twist()
        cmd.linear.x = 0.0
        cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel(cmd)


    def move_x_time(self, moving_time, linear_speed=0.2, angular_speed=0.2):
        
        cmd = Twist()
        cmd.linear.x = linear_speed
        cmd.angular.z = angular_speed
        
        rospy.loginfo("Moving Forwards")
        self.publish_once_in_cmd_vel(cmd)
        time.sleep(moving_time)
        self.stop_bb8()
        rospy.loginfo("######## Finished Moving Forwards")
    
    def move_square(self, side=0.2):
        
        i = 0
        # More Speed, more time to stop
        time_magnitude = side / 0.2
        
        while not self.ctrl_c and i < 4:
            # Move Forwards
            self.move_x_time(moving_time=2.0*time_magnitude, linear_speed=0.2, angular_speed=0.0)
            # Stop
            self.move_x_time(moving_time=4.0, linear_speed=0.0, angular_speed=0.0)
            # Turn, the turning is not afected by the length of the side we want
            self.move_x_time(moving_time=4.0, linear_speed=0.0, angular_speed=0.2)
            # Stop
            self.move_x_time(moving_time=0.1, linear_speed=0.0, angular_speed=0.0)
            
            i += 1
        rospy.loginfo("######## Finished Moving in a Square")
        

            
if __name__ == '__main__':
    rospy.init_node('move_bb8_test', anonymous=True)
    movebb8_object = MoveBB8()
    try:
        movebb8_object.move_square(side=0.6)
    except rospy.ROSInterruptException:
        pass


<p style="background:#3B8F10;color:white;">END **Python File: move_bb8.py** </p>

In [ ]:
while not self.ctrl_c and i < 4:
            # Move Forwards
            self.move_x_time(moving_time=2.0*time_magnitude, linear_speed=0.2, angular_speed=0.0)
            # Stop

We only change the time BB8 is moving forwards to describe a bigger or smaller Square. Of course, without calibration nor closed loop controle, its very difficult to make it create a perfect size square. But the objective is not that. The objective is that the square changes, it doenst have to be exact.

* Create a new python file, called **bb8_move_custom_service_server.py**, modifying the service server that accepts an Empty Service message and activates the square movement that you created in Exercise 3.2. This new service could be called **/move_bb8_in_square_custom**. This new service will have to use service messages of type **BB8CustomServiceMessage** defined here:

The first thing is to create the <a href="#BB8CustomServiceMessage_srv">BB8CustomServiceMessage.srv</a>, creating a **srv** folder inside the package **services_quiz**.

<p style="background:#3B8F10;color:white;" id="BB8CustomServiceMessage_srv">**Service Message: BB8CustomServiceMessage.srv** </p>

In [ ]:
float64 side       # The distance of each side of the square
int32 repetitions    # The number of times BB-8 has to execute the square movement when the service is called
---
bool success         # Did it achieve it?

<p style="background:#3B8F10;color:white;" id="BB8CustomServiceMessage_srv">END **Service Message: BB8CustomServiceMessage.srv** </p>

We have to also modify the <a href="#CMakelists_txt">CMakelists.txt</a> and the <a href="#package_xml">package.xml</a>

<p style="background:#3B8F10;color:white;" id="CMakelists_txt">**CMakelists.txt** </p>

In [ ]:
cmake_minimum_required(VERSION 2.8.3)
project(services_quiz)


## Here go all the packages needed to COMPILE the messages of topic, services and actions.
## Its only geting its paths, and not really importing them to be used in the compilation.
## Its only for further functions in CMakeLists.txt to be able to find those packages.
## In package.xml you have to state them as build
find_package(catkin REQUIRED COMPONENTS
  std_msgs
  message_generation
)

## Generate services in the 'srv' folder
## In this function will be all the action messages of this package ( in the action folder ) to be compiled.
## You can state that it gets all the actions inside the action directory: DIRECTORY action
## Or just the action messages stated explicitly: FILES my_custom_action.action
## In your case you only need to do one of two things, as you wish.
add_service_files(
  FILES
  BB8CustomServiceMessage.srv
)

## Here is where the packages needed for the action messages compilation are imported.
generate_messages(
  DEPENDENCIES
  std_msgs
)

## State here all the packages that will be needed by someone that executes something from your package.
## All the packages stated here must be in the package.xml as exec_depend
catkin_package(
  CATKIN_DEPENDS rospy
)


include_directories(
  ${catkin_INCLUDE_DIRS}
)

<p style="background:#3B8F10;color:white;" id="package_xml">**package.xml** </p>

In [ ]:
<?xml version="1.0"?>
<package format="2">
  <name>services_quiz</name>
  <version>0.0.0</version>
  <description>The services_quiz package</description>

  <maintainer email="user@todo.todo">user</maintainer>

  <license>TODO</license>

  <buildtool_depend>catkin</buildtool_depend>
  <build_depend>rospy</build_depend>
  <build_depend>std_msgs</build_depend>
  <build_depend>message_generation</build_depend>
  <build_export_depend>rospy</build_export_depend>
  <exec_depend>rospy</exec_depend>
  <build_export_depend>std_msgs</build_export_depend>
  <exec_depend>std_msgs</exec_depend>
  <build_export_depend>message_runtime</build_export_depend>
  <exec_depend>message_runtime</exec_depend>

  <export>
  </export>
</package>

Once you have it just compile and source in **ALL the webwhells** that you are going to use so that ROS can find the new Messages.

In [ ]:
roscd;cd ..
catkin_make
source devel/setup.bash

And check it finds the messages:

In [ ]:
rossrv list | grep BB8CustomServiceMessage

You should get:

In [ ]:
services_quiz/BB8CustomServiceMessage

* Use the data passed to this new **/move_bb8_in_square_custom** to change the way BB-8 moves.<br>
Depending on the **side** value, the service must move the BB-8 has to generate a shape of a square based on the **side** given.<br>
Also, the BB-8 must repeat the shape as many times as indicated in the **repetitions** variable of the message.<br> Finally, it must return True if everything went OK in the **success** variable. 

Now its time to create the <a href="#bb8_move_custom_service_server.py">bb8_move_custom_service_server.py</a>, using the new service messages **BB8CustomServiceMessage**.

<p style="background:#3B8F10;color:white;" id="bb8_move_custom_service_server.py">**Python File: bb8_move_custom_service_server.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from services_quiz.srv import BB8CustomServiceMessage, BB8CustomServiceMessageResponse
from move_bb8 import MoveBB8

"""
# BB8CustomServiceMessage
float64 side       # The distance of each side of the square
int32 repetitions    # The number of times BB-8 has to execute the square movement when the service is called
---
bool success         # Did it achieve it?
"""

def my_callback(request):
    rospy.loginfo("The Service move_bb8_in_square_custom has been called")
    
    movebb8_object = MoveBB8()
    
    # We need to add because if we ask 0 repetitions means to execute once, not zero times.
    repetitions_of_square = request.repetitions + 1
    new_side = request.side
    for i in range(repetitions_of_square):
        rospy.loginfo("Start Movement with side = "+str(new_side)+"Repetition = "+str(i))
        movebb8_object.move_square(side=new_side)
        
    rospy.loginfo("Finished service move_bb8_in_square that was called called")
    response = BB8CustomServiceMessageResponse()
    response.success = True
    return response

rospy.init_node('service_move_bb8_in_square_custom_server') 
my_service = rospy.Service('/move_bb8_in_square_custom', BB8CustomServiceMessage , my_callback) # create the Service called move_bb8_in_square with the defined callback
rospy.loginfo("Service /move_bb8_in_square_custom Ready")
rospy.spin() # mantain the service open.

<p style="background:#3B8F10;color:white;">END **Python File: bb8_move_custom_service_server.py** </p>

* Create a new launch called **start_bb8_move_custom_service_server.launch** that launches the server launched in the python file **bb8_move_custom_service_server.py**.
* Test that when calling this new service **/move_bb8_in_square_custom**, BB8 moves accordingly.

This **start_bb8_move_custom_service_server.launch** is the same as the one made in Exercise 3.2, changing the python launched to **bb8_move_custom_service_server.py**.

In [ ]:
<launch>
    <!-- Start Service Server for move_bb8_in_square service -->
    <node pkg="services_quiz" type="bb8_move_custom_service_server.py" name="service_move_bb8_in_square_custom_server"  output="screen">
    </node>
</launch>

* Create a new service client that calls the service **/move_bb8_in_square_custom** and makes BB8 move in a small square **twice** and in a big square **once**. It could be called **bb8_move_custom_service_client.py** and the launch that starts it **call_bb8_move_in_square_custom_service_server.launch**.

We first create the <a href="#bb8_move_custom_service_client_py">bb8_move_custom_service_client.py</a> that will execute a call to perform the two small squares and one big square.

<p style="background:#3B8F10;color:white;" id="bb8_move_custom_service_client_py">**Python File: bb8_move_custom_service_client.py** </p>

In [ ]:
#! /usr/bin/env python
import rospkg
import rospy
from services_quiz.srv import BB8CustomServiceMessage, BB8CustomServiceMessageRequest


rospy.init_node('service_move_bb8_in_square_custom_client') # Initialise a ROS node with the name service_client
rospy.wait_for_service('/move_bb8_in_square_custom') # Wait for the service client /move_bb8_in_square to be running
move_bb8_in_square_service_client = rospy.ServiceProxy('/move_bb8_in_square_custom', BB8CustomServiceMessage) # Create the connection to the service
move_bb8_in_square_request_object = BB8CustomServiceMessageRequest() # Create an object of type EmptyRequest


"""
# BB8CustomServiceMessage
float64 side       # The distance of each side of the square
int32 repetitions    # The number of times BB-8 has to execute the square movement when the service is called
---
bool success         # Did it achieve it?
"""
move_bb8_in_square_request_object.side = 0.1
move_bb8_in_square_request_object.repetitions = 2

rospy.loginfo("Start Two Small Squares...")
result = move_bb8_in_square_service_client(move_bb8_in_square_request_object) # Send through the connection the path to the trajectory file to be executed
rospy.loginfo(str(result)) # Print the result given by the service called

move_bb8_in_square_request_object.side = 0.6
move_bb8_in_square_request_object.repetitions = 1

rospy.loginfo("Start One Big Square...")
result = move_bb8_in_square_service_client(move_bb8_in_square_request_object) # Send through the connection the path to the trajectory file to be executed
rospy.loginfo(str(result))

rospy.loginfo("END of Service call...")

<p style="background:#3B8F10;color:white;">END **Python File: bb8_move_custom_service_client.py** </p>

And now we have to create a launch that launches this python node ,called <a href="#call_bb8_move_in_square_custom_service_server_launch">call_bb8_move_in_square_custom_service_server.launch</a>:

<p style="background:#3B8F10;color:white;" id="call_bb8_move_in_square_custom_service_server_launch">**Launch File: call_bb8_move_in_square_custom_service_server.launch** </p>

In [ ]:
<launch>
    <!-- Start Service Server for move_bb8_in_square service -->
    <node pkg="services_quiz" type="bb8_move_custom_service_client.py" name="service_move_bb8_in_square_custom_client"  output="screen">
    </node>
</launch>

<p style="background:#3B8F10;color:white;">END **Launch File: call_bb8_move_in_square_custom_service_server.launch** </p>

The finally you just have to test all the pipeline, so you have to launch the server launch in one web shell and the client in the other. It should make the robot move as desired.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>
<br>

In [ ]:
roslaunch services_quiz start_bb8_move_custom_service_server.launch

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>
<br>

In [ ]:
roslaunch services_quiz call_bb8_move_in_square_custom_service_server.launch

You should get something similar to this, but slower, because this has been accelerated for practical reasons:

<img src="../img/basic_unit3_exercise3-3_final.gif"/>

## Solution EXTRA Odometry:

Here you have an example of how to access and move based on the odometry readings. Bare in mind that the odometry is not perfect, and therefore there are some inexactitudes, specially in the orientation. The position is quite precise. This is why robots need other localization systems, but thats a very advanced topics. If you are interesetd, please do the **ROS navigation** course or the robot courses of **Jackal robot** or ** Summit XL**.

<p style="background:#3B8F10;color:white;" id="UPGRADE_move_bb8_py">UPGRADE **Python File: move_bb8.py** </p>

In [ ]:
#!/usr/bin/env python
import rospy
from geometry_msgs.msg import Twist
import time
import math
from nav_msgs.msg import Odometry
import tf

"""
rosmsg show nav_msgs/Odometry
std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
string child_frame_id
geometry_msgs/PoseWithCovariance pose
  geometry_msgs/Pose pose
    geometry_msgs/Point position
      float64 x
      float64 y
      float64 z
    geometry_msgs/Quaternion orientation
      float64 x
      float64 y
      float64 z
      float64 w
  float64[36] covariance
geometry_msgs/TwistWithCovariance twist
  geometry_msgs/Twist twist
    geometry_msgs/Vector3 linear
      float64 x
      float64 y
      float64 z
    geometry_msgs/Vector3 angular
      float64 x
      float64 y
      float64 z
  float64[36] covariance
"""


class OdomTools(object):
    def __init__(self):
        self.init_odom()
        self.odom_subscriber = rospy.Subscriber("/odom", Odometry, self.odom_callback)

    def init_odom(self):
        self._odometry = None
        while self._odometry is None:
            try:                
                self._odometry = rospy.wait_for_message("/odom", Odometry, timeout=1)
            except:
                rospy.loginfo("/odom topic is not ready yet, retrying")
        
        rospy.loginfo("/odom topic READY")
        

    def odom_callback(self,msg):
        self._odometry = msg
    
    def get_odom_position(self):
        return self._odometry.pose.pose.position
        
    def get_odom_orientation(self):
        return self._odometry.pose.pose.orientation
    
    def calculate_distance_change(self,init_point, finish_point):
        
        delta_x = finish_point.x - init_point.x
        delta_y = finish_point.y - init_point.y
        delta_z = finish_point.z - init_point.z
        
        delta_x2 = pow(delta_x,2)
        delta_y2 = pow(delta_y,2)
        delta_z2 = pow(delta_z,2)
        
        distance = math.sqrt(delta_x2+delta_y2+delta_z2)
        
        return distance
        
    def calculate_yaw_angle_change(self,init_orientation, finish_orientation):

        explicit_init_quat = [init_orientation.x, init_orientation.y, init_orientation.z, init_orientation.w]
        explicit_finish_quat = [finish_orientation.x, finish_orientation.y, finish_orientation.z, finish_orientation.w]

        init_euler = tf.transformations.euler_from_quaternion(explicit_init_quat)
        finish_euler = tf.transformations.euler_from_quaternion(explicit_finish_quat)

        delta_yaw=finish_euler[2]- init_euler[2]
        
        return delta_yaw

class MoveBB8():
    
    def __init__(self):
        self.odom_object = OdomTools()
        self.bb8_vel_publisher = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.ctrl_c = False
        rospy.on_shutdown(self.shutdownhook)
        self.rate = rospy.Rate(10) # 10hz
    
        
    
    def publish_once_in_cmd_vel(self, cmd):
        """
        This is because publishing in topics sometimes fails teh first time you publish.
        In continuos publishing systems there is no big deal but in systems that publish only
        once it IS very important.
        """
        while not self.ctrl_c:
            connections = self.bb8_vel_publisher.get_num_connections()
            if connections > 0:
                self.bb8_vel_publisher.publish(cmd)
                rospy.loginfo("Cmd Published")
                break
            else:
                self.rate.sleep()
    
    
    def shutdownhook(self):
            # works better than the rospy.is_shut_down()
            self.stop_bb8()
            self.ctrl_c = True

    def stop_bb8(self):
        rospy.loginfo("shutdown time! Stop the robot")
        cmd = Twist()
        cmd.linear.x = 0.0
        cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel(cmd)


    def move_x_time(self, moving_time, linear_speed=0.2, angular_speed=0.2):
        
        cmd = Twist()
        cmd.linear.x = linear_speed
        cmd.angular.z = angular_speed
        
        rospy.loginfo("Moving Forwards")
        self.publish_once_in_cmd_vel(cmd)
        time.sleep(moving_time)
        self.stop_bb8()
        rospy.loginfo("######## Finished Moving Forwards")
    
    def move_square(self, side=0.2):
        
        i = 0
        # More Speed, more time to stop
        time_magnitude = side / 0.2
        
        while not self.ctrl_c and i < 4:
            # Move Forwards
            self.move_x_time(moving_time=2.0*time_magnitude, linear_speed=0.2, angular_speed=0.0)
            # Stop
            self.move_x_time(moving_time=4.0, linear_speed=0.0, angular_speed=0.0)
            # Turn, the turning is not afected by the length of the side we want
            self.move_x_time(moving_time=4.0, linear_speed=0.0, angular_speed=0.2)
            # Stop
            self.move_x_time(moving_time=0.1, linear_speed=0.0, angular_speed=0.0)
            
            i += 1
        rospy.loginfo("######## Finished Moving in a Square")
        
    def move_distance(self, distance_to_move,linear_speed=0.2):
        
        cmd = Twist()
        cmd.linear.x = linear_speed
        cmd.angular.z = 0.0
        
        
        init_position = self.odom_object.get_odom_position()
        final_position = self.odom_object.get_odom_position()
        distance_moved = self.odom_object.calculate_distance_change(init_position,final_position)
        
        while distance_moved < distance_to_move:
            rospy.loginfo("Moving")
            self.publish_once_in_cmd_vel(cmd)
            self.rate.sleep()
            final_position = self.odom_object.get_odom_position()
            distance_moved = self.odom_object.calculate_distance_change(init_position,final_position)
            rospy.loginfo("Distance Moved::>"+str(distance_moved))
        
        rospy.loginfo("Stopping...")
        cmd.linear.x = 0.0
        cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel(cmd)
        
    def move_angle(self, angle_to_turn, angular_speed=0.2):
        
        cmd = Twist()
        cmd.linear.x = 0.0
        cmd.angular.z = angular_speed
        
        init_orientation = self.odom_object.get_odom_orientation()
        finish_orientation = self.odom_object.get_odom_orientation()
        delta_yaw = self.odom_object.calculate_yaw_angle_change(init_orientation, finish_orientation)
        
        while delta_yaw < angle_to_turn:
            rospy.loginfo("Turning")
            self.publish_once_in_cmd_vel(cmd)
            self.rate.sleep()
            finish_orientation = self.odom_object.get_odom_orientation()
            delta_yaw = self.odom_object.calculate_yaw_angle_change(init_orientation, finish_orientation)
            rospy.loginfo("Angle Moved::>"+str(delta_yaw))
        
        rospy.loginfo("Stopping...")
        cmd.linear.x = 0.0
        cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel(cmd)
    
    def calibrate_turning(self):
        
        init_orientation = self.odom_object.get_odom_orientation()
        while not self.ctrl_c:
            
            finish_orientation = self.odom_object.get_odom_orientation()
            delta_yaw = self.odom_object.calculate_yaw_angle_change(init_orientation, finish_orientation)
            rospy.loginfo("Angle Moved::>"+str(delta_yaw))
            self.rate.sleep()
        
        
            
if __name__ == '__main__':
    rospy.init_node('move_bb8_test', anonymous=True)
    movebb8_object = MoveBB8()
    try:
        #movebb8_object.move_square(side=0.6)
        #movebb8_object.move_cicle(radius=20.0)
        movebb8_object.move_distance(distance_to_move=1.0,linear_speed=0.2)
        
    except rospy.ROSInterruptException:
        pass


<p style="background:#3B8F10;color:white;" >END UPGRADE **Python File: move_bb8.py** </p>

As you can see its a next step in movement. Of ocurse this could be better if implemented a PID controle to get to the exact position. and orientation. But thats also another topic. If you are interested, please do the course on **Perception** where we talk about using the **pid** ros package for this purpose.